In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
import os
import cv2
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")
# convert_png2raw_reshape_alln1.ipynb

In [2]:
# I) load dataset =============================================================
# pubgTestroot = '/local/mnt3/workspace3/Kishore/Battlemodes_MNIST/data/Test_All_10Nov/'
pubgTestroot = '/local/mnt3/workspace3/Kishore/Battlemodes_MNIST/BattleModeDetection/Nov15Update/MNIST_DATASET/TEST_MNIST_BattleModes/'

#Synthetic Train data sets
# pubgTrainroot='/local/mnt3/workspace3/Kishore/ML_project_dir/ML_project_env/Normalized_NoPreprocessing/Dataset/SyntheticData/'
pubgTrainroot='/local/mnt3/workspace3/Kishore/Battlemodes_MNIST/BattleModeDetection/Nov15Update/MNIST_DATASET/SyntheticData/'

#Real Train dataset
#Here all battle modes (TTB, CCB, TCB battle game data added into one folder)
# pubgTrainroot_realDB='/local/mnt3/workspace3/Kishore/Battlemodes_MNIST/data/Train_real_data/Train_realrename_10Nov/'
pubgTrainroot_realDB='/local/mnt3/workspace3/Kishore/Battlemodes_MNIST/BattleModeDetection/Nov15Update/MNIST_DATASET/Train_realrename_11Nov/'



In [3]:
# load Test data---------------------------------------------------------------
#pytorch model= (imgs,1,28,28) and tensorflow model= (imgs,28,28,1)

Testfilelist= os.listdir(pubgTestroot)
print('load Test Data')
x_test_pubg= np.zeros([len(Testfilelist),1,14,24])
y_test_pubg= np.zeros([len(Testfilelist),1])
for imgindex in range(len(Testfilelist)):     
#    print(imgindex)
    imgcrop  = cv2.imread(pubgTestroot+Testfilelist[imgindex],0)
    # imgcropscaled = cv2.resize(imgcrop, (24,14))
    # imgcrop = np.float32(imgcrop)+np.random.rand(1)[0]/1000
    imgcrop= np.float32(imgcrop) # np.max(imgcrop)
    x_test_pubg[imgindex,0,:,:]=imgcrop
    label = Testfilelist[imgindex][0:3]
    spindex= label.find('_')
    if spindex>0:
         label=label[:spindex]
    y_test_pubg[imgindex,0]= int(label)

x_test_pubg = torch.from_numpy(x_test_pubg) # get torch tensor from numpy array
y_test_pubg = torch.from_numpy(y_test_pubg)
print(x_test_pubg.shape, y_test_pubg.shape)

load Test Data
torch.Size([3000, 1, 14, 24]) torch.Size([3000, 1])


In [4]:
# Load Synthetic trainig data ----------------------------------------
print('load Synthetic Training  Data')
filelist= os.listdir(pubgTrainroot)
np.random.shuffle((filelist))
x_train_pubg= np.zeros([len(filelist),1,14,24])
y_train_pubg= np.zeros([len(filelist),1])
for imgindex in range(len(filelist)):
    
    imgcrop= cv2.imread(pubgTrainroot+filelist[imgindex],0)
    # imgcrop = np.float32(imgcrop)+np.random.rand(1)[0]/1000
    # imgcrop= imgcrop/ 255 # np.max(imgcrop)
    imgcrop= np.float32(imgcrop)
    x_train_pubg[imgindex,0,:,:]= imgcrop
    label = filelist[imgindex][0:3]
    spindex= label.find('_')
    if spindex>0:
        label=label[:spindex]
    y_train_pubg[imgindex,0]= int(label)
# x_train_pubg=x_train_pubg/np.max(x_train_pubg)
print(x_train_pubg.shape, y_train_pubg.shape)

load Synthetic Training  Data
(34566, 1, 14, 24) (34566, 1)


In [5]:
# load Real Data --------------
print('load Real Training  Data')
filelist_realDB= os.listdir(pubgTrainroot_realDB)
x_train_pubg_realDB= np.zeros([len(filelist_realDB),1,14,24])
y_train_pubg_realDB= np.zeros([len(filelist_realDB),1])
for imgindex in range(len(filelist_realDB)):
     
    imgcrop= cv2.imread(pubgTrainroot_realDB+filelist_realDB[imgindex],0)
    # imgcrop = np.float32(imgcrop)+np.random.rand(1)[0]/1000

    # if 	 np.max(imgcrop)<=0:
    # print(filelist_realDB[imgindex] +'    max : '+ str(np.max(imgcrop)))          

    # imgcrop= imgcrop/ 255 # np.max(imgcrop)
    imgcrop= np.float32(imgcrop)
    x_train_pubg_realDB[imgindex,0,:,:]= imgcrop
    label = filelist_realDB[imgindex][0:3]
    spindex= label.find('_')
    if spindex>0:
        label=label[:spindex]
    y_train_pubg_realDB[imgindex,0]= int(label)
# x_train_pubg_realDB=x_train_pubg_realDB/np.max(x_train_pubg_realDB)
print(x_train_pubg_realDB.shape, y_train_pubg_realDB.shape)

load Real Training  Data
(25833, 1, 14, 24) (25833, 1)


In [6]:
# =============================================================================
# Define MNIST MODEL ----------------------------------------------------------
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3)
        # nn.init.xavier_normal_(self.conv1.weight)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3)
        # nn.init.xavier_normal_(self.conv2.weight)
        self.drop1 = nn.Dropout2d(0.3)
        self.drop2 = nn.Dropout(0.5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(800, 64)
        # nn.init.xavier_normal_(self.fc1.weight)
        self.fc2 = nn.Linear(64, 302)
        # nn.init.xavier_normal_(self.fc2.weight)

    def forward(self, x):
        x = torch.div(x,255)
        
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(F.relu(self.conv2(x)) , 2)
        x= self.drop1(x)
        x= self.flatten(x)
        x= self.fc1(x)
        x= self.drop2(x)
        x= self.fc2(x)
        # return F.log_softmax(x, dim=1)
        return x
        

In [7]:
# define loss function and run training ---------------------------------------

# n_epochs = 3
batch_size_train = 16
# batch_size_test = 1000
learning_rate = 0.005
momentum = 0.4
# log_interval = 10

random_seed = 1

torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [8]:
network = Net()
network = network.float() 
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
# optimizer = optim.ADM(network.parameters(), lr=learning_rate) 
# optimizer =optim.Adam(network.parameters(),lr=learning_rate) 

criterion = nn.CrossEntropyLoss()
# Training --------------------------------------------------------------------

testaccset=[]
testaccset_top=[]
maxacc=0.99
for epoch_index in range(1000):
    
    if epoch_index>0 and epoch_index% 300 ==0:
        learning_rate=learning_rate*0.1
        print('learning_rate',learning_rate)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)

        
    num_random_sample = len(x_train_pubg_realDB)
    random_index= np.random.choice(np.arange(len(x_train_pubg)),num_random_sample )

    train_random_y = y_train_pubg[random_index,:]    
    train_random_x = x_train_pubg[random_index,:,:,:]    
    
    
    
    temp_data_x= np.concatenate((train_random_x, x_train_pubg_realDB) , axis=0 )
    temp_data_y= np.concatenate((train_random_y, y_train_pubg_realDB) , axis=0 )
    
    index_shuffle= np.arange(temp_data_y.shape[0])
    np.random.shuffle(index_shuffle[:])
    
    temp_data_x=temp_data_x[index_shuffle[:],:,:,:]
    temp_data_y=temp_data_y[index_shuffle[:],:]

    temp_data_x_torch = torch.from_numpy(temp_data_x)
    temp_data_y_torch = torch.from_numpy(temp_data_y)

    numbatch = temp_data_x_torch.shape[0]//batch_size_train
    
    network.train()
    for batch_ind in range(numbatch):

        temp_data_x_torch_batch = temp_data_x_torch[batch_ind * batch_size_train : (batch_ind+1) * batch_size_train,:]
        temp_data_y_torch_batch = temp_data_y_torch[batch_ind * batch_size_train : (batch_ind+1) * batch_size_train]
    # reset optimizer grad --------------------------
        optimizer.zero_grad()
        output = network(temp_data_x_torch_batch.float())
        # loss = F.nll_loss(output, temp_data_y_torch_batch[:,0].long())
        loss= criterion(output ,temp_data_y_torch_batch[:,0].long() ) 
        loss.backward()
        optimizer.step()
        
        # accuracy = (torch.argmax(output, dim=1) == temp_data_y_torch_batch[:,0]).sum().item() / batch_size_train
        # print( f'batch { batch_ind} , accuracy {accuracy} ')
    with torch.no_grad():
        network.eval()
        output_test = network(x_test_pubg.float())
        accuracy4Test =(torch.argmax(output_test, dim=1) == y_test_pubg[:,0]).to(torch.float).mean()
        
        output_train = network(torch.from_numpy(x_train_pubg_realDB).float())
        accuracy4Train =(torch.argmax(output_train, dim=1) == torch.from_numpy(y_train_pubg_realDB)[:,0]).to(torch.float).mean()

        output_train2 = network(torch.from_numpy(x_train_pubg).float())
        accuracy4Train2 =(torch.argmax(output_train2, dim=1) == torch.from_numpy(y_train_pubg)[:,0]).to(torch.float).mean()
        
        print( f'epoch_index { epoch_index} , accuracy4TrainSynthetic {accuracy4Train2},accuracy4Train {accuracy4Train} , accuracy4Test {accuracy4Test} ')
        outpath = "/local/mnt3/workspace3/Kishore/Battlemodes_MNIST/"
        if (accuracy4Test and epoch_index >400) >=maxacc:
        #if epoch_index %250 == 0:
            torch.save(network.state_dict(), outpath+ f'/chkpoints/Dec22_accTrainSynthetic_{accuracy4Train2:.4f}_accTrain_{accuracy4Train:.4f}_accTest_{accuracy4Test:.4f}_iter_{epoch_index}_model.pt') 
            torch.save(optimizer.state_dict(), outpath+ f'/chkpoints/Dec22_accTrainSynthetic_{accuracy4Train2:.4f}_accTrain_{accuracy4Train:.4f}_accTest_{accuracy4Test:.4f}_iter_{epoch_index}_optimizer.pt')
            maxacc=accuracy4Test.item()


epoch_index 0 , accuracy4TrainSynthetic 0.034918706864118576,accuracy4Train 0.18046684563159943 , accuracy4Test 0.12666666507720947 
epoch_index 1 , accuracy4TrainSynthetic 0.08872880786657333,accuracy4Train 0.3836565613746643 , accuracy4Test 0.2930000126361847 
epoch_index 2 , accuracy4TrainSynthetic 0.29445120692253113,accuracy4Train 0.6050787568092346 , accuracy4Test 0.6050000190734863 
epoch_index 3 , accuracy4TrainSynthetic 0.5742637515068054,accuracy4Train 0.7764100432395935 , accuracy4Test 0.7770000100135803 
epoch_index 4 , accuracy4TrainSynthetic 0.7021350264549255,accuracy4Train 0.8446173667907715 , accuracy4Test 0.8456666469573975 
epoch_index 5 , accuracy4TrainSynthetic 0.7770641446113586,accuracy4Train 0.8889405131340027 , accuracy4Test 0.9266666769981384 
epoch_index 6 , accuracy4TrainSynthetic 0.8158016800880432,accuracy4Train 0.9105020761489868 , accuracy4Test 0.9696666598320007 
epoch_index 7 , accuracy4TrainSynthetic 0.8379332423210144,accuracy4Train 0.931482970714569

epoch_index 64 , accuracy4TrainSynthetic 0.9850431084632874,accuracy4Train 0.9936902523040771 , accuracy4Test 0.9916666746139526 
epoch_index 65 , accuracy4TrainSynthetic 0.9860556721687317,accuracy4Train 0.9940386414527893 , accuracy4Test 0.9893333315849304 
epoch_index 66 , accuracy4TrainSynthetic 0.9851588010787964,accuracy4Train 0.9936515092849731 , accuracy4Test 0.9906666874885559 
epoch_index 67 , accuracy4TrainSynthetic 0.9854770302772522,accuracy4Train 0.9940386414527893 , accuracy4Test 0.9916666746139526 
epoch_index 68 , accuracy4TrainSynthetic 0.9865474700927734,accuracy4Train 0.9941547513008118 , accuracy4Test 0.9913333058357239 
epoch_index 69 , accuracy4TrainSynthetic 0.9865474700927734,accuracy4Train 0.9935740828514099 , accuracy4Test 0.9916666746139526 
epoch_index 70 , accuracy4TrainSynthetic 0.9864895939826965,accuracy4Train 0.9948515295982361 , accuracy4Test 0.9909999966621399 
epoch_index 71 , accuracy4TrainSynthetic 0.9859977960586548,accuracy4Train 0.9940386414527

epoch_index 127 , accuracy4TrainSynthetic 0.993548572063446,accuracy4Train 0.9961289763450623 , accuracy4Test 0.9916666746139526 
epoch_index 128 , accuracy4TrainSynthetic 0.9935774803161621,accuracy4Train 0.9962838292121887 , accuracy4Test 0.9916666746139526 
epoch_index 129 , accuracy4TrainSynthetic 0.993548572063446,accuracy4Train 0.9962838292121887 , accuracy4Test 0.9916666746139526 
epoch_index 130 , accuracy4TrainSynthetic 0.9927674531936646,accuracy4Train 0.9964386820793152 , accuracy4Test 0.9916666746139526 
epoch_index 131 , accuracy4TrainSynthetic 0.9938957095146179,accuracy4Train 0.9965934753417969 , accuracy4Test 0.9943333268165588 
epoch_index 132 , accuracy4TrainSynthetic 0.993606448173523,accuracy4Train 0.9958580136299133 , accuracy4Test 0.9916666746139526 
epoch_index 133 , accuracy4TrainSynthetic 0.9938957095146179,accuracy4Train 0.996322512626648 , accuracy4Test 0.9916666746139526 
epoch_index 134 , accuracy4TrainSynthetic 0.993548572063446,accuracy4Train 0.9959354400

epoch_index 190 , accuracy4TrainSynthetic 0.9956604838371277,accuracy4Train 0.9971741437911987 , accuracy4Test 0.9916666746139526 
epoch_index 191 , accuracy4TrainSynthetic 0.995197594165802,accuracy4Train 0.997290313243866 , accuracy4Test 0.9936666488647461 
epoch_index 192 , accuracy4TrainSynthetic 0.9958919286727905,accuracy4Train 0.9968644976615906 , accuracy4Test 0.9916666746139526 
epoch_index 193 , accuracy4TrainSynthetic 0.9960654973983765,accuracy4Train 0.9970967173576355 , accuracy4Test 0.9916666746139526 
epoch_index 194 , accuracy4TrainSynthetic 0.9953711628913879,accuracy4Train 0.997251570224762 , accuracy4Test 0.9916666746139526 
epoch_index 195 , accuracy4TrainSynthetic 0.9958919286727905,accuracy4Train 0.9972128868103027 , accuracy4Test 0.9909999966621399 
epoch_index 196 , accuracy4TrainSynthetic 0.9954869151115417,accuracy4Train 0.9973289966583252 , accuracy4Test 0.9916666746139526 
epoch_index 197 , accuracy4TrainSynthetic 0.9961233735084534,accuracy4Train 0.99717414

epoch_index 253 , accuracy4TrainSynthetic 0.9970780611038208,accuracy4Train 0.9970967173576355 , accuracy4Test 0.9943333268165588 
epoch_index 254 , accuracy4TrainSynthetic 0.996788740158081,accuracy4Train 0.9975225329399109 , accuracy4Test 0.9943333268165588 
epoch_index 255 , accuracy4TrainSynthetic 0.9968177080154419,accuracy4Train 0.9970580339431763 , accuracy4Test 0.9916666746139526 
epoch_index 256 , accuracy4TrainSynthetic 0.996673047542572,accuracy4Train 0.9972128868103027 , accuracy4Test 0.9923333525657654 
epoch_index 257 , accuracy4TrainSynthetic 0.9971359372138977,accuracy4Train 0.9973676800727844 , accuracy4Test 0.9916666746139526 
epoch_index 258 , accuracy4TrainSynthetic 0.9965572953224182,accuracy4Train 0.9977161288261414 , accuracy4Test 0.9933333396911621 
epoch_index 259 , accuracy4TrainSynthetic 0.9970491528511047,accuracy4Train 0.997251570224762 , accuracy4Test 0.9919999837875366 
epoch_index 260 , accuracy4TrainSynthetic 0.9970780611038208,accuracy4Train 0.99752253

...

epoch_index 695 , accuracy4TrainSynthetic 0.998235285282135,accuracy4Train 0.998180627822876 , accuracy4Test 0.9923333525657654 
epoch_index 696 , accuracy4TrainSynthetic 0.9981774091720581,accuracy4Train 0.9982193112373352 , accuracy4Test 0.9923333525657654 
epoch_index 697 , accuracy4TrainSynthetic 0.9982063174247742,accuracy4Train 0.9982193112373352 , accuracy4Test 0.9923333525657654 
epoch_index 698 , accuracy4TrainSynthetic 0.9982063174247742,accuracy4Train 0.9982193112373352 , accuracy4Test 0.9923333525657654 
epoch_index 699 , accuracy4TrainSynthetic 0.998235285282135,accuracy4Train 0.998180627822876 , accuracy4Test 0.9923333525657654 
epoch_index 700 , accuracy4TrainSynthetic 0.998235285282135,accuracy4Train 0.9982193112373352 , accuracy4Test 0.9923333525657654 
epoch_index 701 , accuracy4TrainSynthetic 0.998235285282135,accuracy4Train 0.9982193112373352 , accuracy4Test 0.9923333525657654 
epoch_index 702 , accuracy4TrainSynthetic 0.9982063174247742,accuracy4Train 0.99818062782

epoch_index 759 , accuracy4TrainSynthetic 0.998235285282135,accuracy4Train 0.998180627822876 , accuracy4Test 0.9923333525657654 
epoch_index 760 , accuracy4TrainSynthetic 0.9982641935348511,accuracy4Train 0.998180627822876 , accuracy4Test 0.9923333525657654 
epoch_index 761 , accuracy4TrainSynthetic 0.998235285282135,accuracy4Train 0.9982193112373352 , accuracy4Test 0.9923333525657654 
epoch_index 762 , accuracy4TrainSynthetic 0.998235285282135,accuracy4Train 0.9982193112373352 , accuracy4Test 0.9923333525657654 
epoch_index 763 , accuracy4TrainSynthetic 0.998235285282135,accuracy4Train 0.998180627822876 , accuracy4Test 0.9923333525657654 
epoch_index 764 , accuracy4TrainSynthetic 0.998235285282135,accuracy4Train 0.9982193112373352 , accuracy4Test 0.9923333525657654 
epoch_index 765 , accuracy4TrainSynthetic 0.9982641935348511,accuracy4Train 0.998180627822876 , accuracy4Test 0.9923333525657654 
epoch_index 766 , accuracy4TrainSynthetic 0.998235285282135,accuracy4Train 0.998180627822876

epoch_index 823 , accuracy4TrainSynthetic 0.9982641935348511,accuracy4Train 0.998141884803772 , accuracy4Test 0.9923333525657654 
epoch_index 824 , accuracy4TrainSynthetic 0.9982931017875671,accuracy4Train 0.998141884803772 , accuracy4Test 0.9923333525657654 
epoch_index 825 , accuracy4TrainSynthetic 0.9982931017875671,accuracy4Train 0.998141884803772 , accuracy4Test 0.9923333525657654 
epoch_index 826 , accuracy4TrainSynthetic 0.998235285282135,accuracy4Train 0.998141884803772 , accuracy4Test 0.9923333525657654 
epoch_index 827 , accuracy4TrainSynthetic 0.998235285282135,accuracy4Train 0.998180627822876 , accuracy4Test 0.9919999837875366 
epoch_index 828 , accuracy4TrainSynthetic 0.9982931017875671,accuracy4Train 0.998180627822876 , accuracy4Test 0.9923333525657654 
epoch_index 829 , accuracy4TrainSynthetic 0.9982641935348511,accuracy4Train 0.998180627822876 , accuracy4Test 0.9923333525657654 
epoch_index 830 , accuracy4TrainSynthetic 0.9982641935348511,accuracy4Train 0.99818062782287

epoch_index 886 , accuracy4TrainSynthetic 0.9982641935348511,accuracy4Train 0.9982193112373352 , accuracy4Test 0.9919999837875366 
epoch_index 887 , accuracy4TrainSynthetic 0.998235285282135,accuracy4Train 0.998141884803772 , accuracy4Test 0.9916666746139526 
epoch_index 888 , accuracy4TrainSynthetic 0.9982641935348511,accuracy4Train 0.998180627822876 , accuracy4Test 0.9919999837875366 
epoch_index 889 , accuracy4TrainSynthetic 0.9982641935348511,accuracy4Train 0.9982193112373352 , accuracy4Test 0.9923333525657654 
epoch_index 890 , accuracy4TrainSynthetic 0.9982641935348511,accuracy4Train 0.998180627822876 , accuracy4Test 0.9923333525657654 
epoch_index 891 , accuracy4TrainSynthetic 0.9982931017875671,accuracy4Train 0.998141884803772 , accuracy4Test 0.9923333525657654 
epoch_index 892 , accuracy4TrainSynthetic 0.9982641935348511,accuracy4Train 0.998180627822876 , accuracy4Test 0.9923333525657654 
epoch_index 893 , accuracy4TrainSynthetic 0.9982641935348511,accuracy4Train 0.99818062782

epoch_index 949 , accuracy4TrainSynthetic 0.998235285282135,accuracy4Train 0.998180627822876 , accuracy4Test 0.9919999837875366 
epoch_index 950 , accuracy4TrainSynthetic 0.9982641935348511,accuracy4Train 0.998180627822876 , accuracy4Test 0.9919999837875366 
epoch_index 951 , accuracy4TrainSynthetic 0.9982641935348511,accuracy4Train 0.998180627822876 , accuracy4Test 0.9919999837875366 
epoch_index 952 , accuracy4TrainSynthetic 0.9982641935348511,accuracy4Train 0.998180627822876 , accuracy4Test 0.9919999837875366 
epoch_index 953 , accuracy4TrainSynthetic 0.9982641935348511,accuracy4Train 0.9982193112373352 , accuracy4Test 0.9919999837875366 
epoch_index 954 , accuracy4TrainSynthetic 0.9982641935348511,accuracy4Train 0.998180627822876 , accuracy4Test 0.9919999837875366 
epoch_index 955 , accuracy4TrainSynthetic 0.998235285282135,accuracy4Train 0.998180627822876 , accuracy4Test 0.9919999837875366 
epoch_index 956 , accuracy4TrainSynthetic 0.9982641935348511,accuracy4Train 0.9981806278228

In [ ]:
#This model is no argmax Pl. run: MNIST_noargmax_argmax_pt.ipynb